In [1]:
import os

In [2]:
data_path = "./latexData"


In [3]:
# print each directory and file in the data_path

for root, dirs, files in os.walk(data_path):
    print(f"Directory: {root}")
    for file in files:
        print(f"  File: {file}")
    for dir in dirs:
        print(f"  Subdirectory: {dir}")
    print()

Directory: ./latexData
  Subdirectory: nodescore

Directory: ./latexData\nodescore
  File: nodescore_without_broadcast.csv
  File: nodescore_with_broadcast.csv



# Node Score Table

In [4]:
import pandas as pd
import os

In [8]:
# Read CSV files
without_broadcast = pd.read_csv(os.path.join(data_path, "nodescore", "nodescore_without_broadcast.csv"))
with_broadcast = pd.read_csv(os.path.join(data_path, "nodescore", "nodescore_with_broadcast.csv"))

In [9]:
without_broadcast.head()

,method,D1,D2,M,S,p1,p2,p3,p4,p5,p6,p7
0,Betweenness Centrality,0.000000,0.000000,1.000000,0.000000,0.131579,0.236842,0.105263,0.087719,0.087719,0.087719,0.052632
1,Closeness Centrality,0.720614,1.000000,0.701091,0.000000,0.624769,0.624769,0.624769,0.110290,0.110290,0.110290,0.110290
2,Communication Path Centrality,0.215075,0.315645,1.000000,0.149935,0.434269,0.500823,0.402972,0.446720,0.446720,0.446720,0.336399
3,Harmonic Communication Centrality,0.374199,0.549176,1.000000,0.260865,0.576988,0.602212,0.576988,0.562623,0.562623,0.562623,0.428855
4,Degree Centrality,0.333333,0.500000,1.000000,0.666667,0.333333,0.500000,0.333333,0.333333,0.333333,0.333333,0.333333


In [10]:
with_broadcast.head()

,method,B,D1,D2,M,S,p1,p2,p3,p4,p5,p6,p7
0,Betweenness Centrality,1.000000,0.000000,0.000000,0.263889,0.000000,0.034722,0.062500,0.027778,0.023148,0.023148,0.023148,0.013889
1,Closeness Centrality,1.000000,0.693090,0.864099,0.579348,0.393627,0.626178,0.626178,0.626178,0.424516,0.424516,0.424516,0.424516
2,Communication Path Centrality,0.116195,0.211774,0.313772,1.000000,0.155797,0.417272,0.489417,0.401251,0.445825,0.445825,0.445825,0.332924
3,Harmonic Communication Centrality,0.162958,0.368524,0.546020,1.000000,0.271115,0.573453,0.606797,0.573428,0.560527,0.560527,0.560527,0.420610
4,Degree Centrality,1.000000,0.583333,0.604167,0.666667,0.625000,0.583333,0.604167,0.583333,0.583333,0.583333,0.583333,0.583333


In [20]:

# Get methods from without_broadcast as reference
methods = without_broadcast['method'].tolist()

method_replacements = {
    'Communication Path Centrality': 'CPC',
    'Harmonic Communication Centrality': 'HCC',
}

bold_methods = {
    'Communication Path Centrality',
    'Harmonic Communication Centrality',
}
    

# Define nodes in the desired order
nodes = ['D2', 'p3', 'p7', 'D1', 'p2', 'p1', 'M', 'p6', 'p5', 'p4', 'S', 'B']


In [ ]:

# Generate LaTeX table
latex = []
# latex.append("\\begin{table}")
# latex.append("\\centering")
# latex.append("\\caption{Node Score Comparison}")
# latex.append("\\label{tab:node-score}")

# Table structure
# Create column format: l for first column, then cc| for each node except B (which gets only c|)
column_format = "|l|"
for node in nodes:
    if node == 'B':
        column_format += "c|"  # B only needs the 'w/ B' column
    else:
        column_format += "cc|"  # Other nodes need both 'w/o B' and 'w/ B'
latex.append(f"\\begin{{tabular}}{{{column_format}}}")
latex.append("\\toprule")

# First header row with nodes
header1 = ["\\multirow{2}{*}{Centrality Method}"]
for node in nodes:
    if node == 'B':
        header1.append(f"\\multicolumn{{1}}{{c|}}{{{node}}}")
    else:
        header1.append(f"\\multicolumn{{2}}{{c|}}{{{node}}}")
latex.append(" & ".join(header1) + " \\\\")

# Second header row with w/o B and w/ B
header2 = [""]
for node in nodes:
    if node == 'B':
        header2.append("w/ B")
    else:
        header2.append("w/o B")
        header2.append("w/ B")
latex.append(" & ".join(header2) + " \\\\")
latex.append("\\midrule")

# Add data rows
for method in methods:
    # Replace "Centrality" by ""
    method_name = method.replace("Centrality", "").strip()
    if method in method_replacements:
        method_name = method_replacements[method]
    
    # Check if this is a method that should be bold
    is_bold = method in bold_methods
    
    if is_bold:
        row = [f"\\textbf{{{method_name}}}"]
    else:
        row = [method_name]
    # row = [method_name]
    
    # Get row data from both dataframes
    without_row = without_broadcast[without_broadcast['method'] == method].iloc[0]
    with_row = with_broadcast[with_broadcast['method'] == method].iloc[0]
    

    
    for node in nodes:
        if node == 'B':
            # For B, only include the 'w/ B' value
            if node in with_broadcast.columns:
                value = with_row[node]
                if pd.notna(value) and isinstance(value, (float, int)):
                    cell = f"{value:.2f}"
                    if is_bold:
                        cell = f"\\textbf{{{cell}}}"
                    row.append(cell)
                else:
                    row.append("-")
            else:
                row.append("-")
        else:
            # For without_broadcast
            if node in without_broadcast.columns:
                value = without_row[node]
                if pd.notna(value) and isinstance(value, (float, int)):
                    cell = f"{value:.2f}"
                    if is_bold:
                        cell = f"\\textbf{{{cell}}}"
                    row.append(cell)
                else:
                    row.append("-")
            else:
                row.append("-")
            
            # For with_broadcast
            if node in with_broadcast.columns:
                value = with_row[node]
                if pd.notna(value) and isinstance(value, (float, int)):
                    cell = f"{value:.2f}"
                    if is_bold:
                        cell = f"\\textbf{{{cell}}}"
                    row.append(cell)
                else:
                    row.append("-")
            else:
                row.append("-")
    
    latex.append(" & ".join(row) + " \\\\")

latex.append("\\bottomrule")
latex.append("\\end{tabular}")
# latex.append("\\end{table}")

# Print the full LaTeX table code
print("% Requires packages: \\usepackage{booktabs} \\usepackage{multirow}")
print("\n".join(latex))

% Requires packages: \usepackage{booktabs} \usepackage{multirow}
\begin{tabular}{l|cc|cc|cc|cc|cc|cc|cc|cc|cc|cc|cc|c|}
\toprule
\multirow{2}{*}{Centrality Method} & \multicolumn{2}{c|}{D2} & \multicolumn{2}{c|}{p3} & \multicolumn{2}{c|}{p7} & \multicolumn{2}{c|}{D1} & \multicolumn{2}{c|}{p2} & \multicolumn{2}{c|}{p1} & \multicolumn{2}{c|}{M} & \multicolumn{2}{c|}{p6} & \multicolumn{2}{c|}{p5} & \multicolumn{2}{c|}{p4} & \multicolumn{2}{c|}{S} & \multicolumn{1}{c|}{B} \\
 & w/o B & w/ B & w/o B & w/ B & w/o B & w/ B & w/o B & w/ B & w/o B & w/ B & w/o B & w/ B & w/o B & w/ B & w/o B & w/ B & w/o B & w/ B & w/o B & w/ B & w/o B & w/ B & w/ B \\
\midrule
Betweenness & 0.00 & 0.00 & 0.11 & 0.03 & 0.05 & 0.01 & 0.00 & 0.00 & 0.24 & 0.06 & 0.13 & 0.03 & 1.00 & 0.26 & 0.09 & 0.02 & 0.09 & 0.02 & 0.09 & 0.02 & 0.00 & 0.00 & 1.00 \\
Closeness & 1.00 & 0.86 & 0.62 & 0.63 & 0.11 & 0.42 & 0.72 & 0.69 & 0.62 & 0.63 & 0.62 & 0.63 & 0.70 & 0.58 & 0.11 & 0.42 & 0.11 & 0.42 & 0.11 & 0.42 & 0.00 & 0.39